# Predicting how points end in tennis

## Abstract

This is part of the code that I used in my solution to the CrowdAnalytix competition. It uses a three-layer neural network to predict the outcome of a tennis points among three classes (Winner, Forced error, Unforced error). The described solution is very raw and I think many improvements could still be made to improve the accuracy of the model (better feature engineering/model ensembling). My final model achieved an accuracy around 90%.

## Motivation
Tennis, one of the most popular professional sports around the world, still uses manual coding of point outcomes.  This is not only labor-intensive but it also raises concerns that outcome categories may not always be consistent from one coder to the next. The purpose of this contest is to find a better approach. 

## Point Endings
Every tennis match is made up of a sequence of points. A point begins with a serve and players exchange shots until a player makes an error or is unable to return a shot in play. 

Traditionally, the shot ending a point in tennis has been had been described in one of three mutually exclusive ways: a winner, an unforced error, or a forced error. A winner is a shot that was in play, not touched by the opponent, and ends with the point going to the player who made the shot. The other two categories are two distinct types of errors where both end with the point going to the player who did not make the shot. The distinction between an unforced and forced error is based on the nature of the incoming shot and a judgment about whether the shot was playable or not. As you can imagine, this distinction is not a perfect science.  

## Outcome Coding
Point endings give us insight into player performance. For this reason, accurate statistics about point outcomes are essential to the sport. At professional tennis tournaments, human coders are trained to label and document outcomes during matches. This is the primary way that the sport gathers information about winners and errors. 

## Tracking Data
The adoption of the player challenge system in the mid-2000s has lead to the use of multi-camera tracking systems for the majority of top professional matches. These tracking systems monitor the 3D coordinates of the ball position and 2D coordinates of the player position throughout a match. The richness of these data hold considerable promise for addressing many challenging questions in the sport.

## Objective

The objective of this contest is as follows:

* Predict how a point ends in tennis using modern tracking data.

In [1]:
import warnings; warnings.simplefilter('ignore')

from time import time
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


## Data

In [2]:
# Train data.
df_mens = pd.read_csv('data/mens_train_file.csv', sep=',',header=0)
df_womens = pd.read_csv('data/womens_train_file.csv', sep=',',header=0)
frames = [df_mens, df_womens]
df = pd.concat(frames)

In [3]:
print(df.head())

   rally  serve hitpoint      speed  net.clearance  distance.from.sideline  \
0      4      1        B  35.515042      -0.021725                3.474766   
1      4      2        B  33.382640       1.114202                2.540801   
2     23      1        B  22.316690      -0.254046                3.533166   
3      9      1        F  36.837309       0.766694                0.586885   
4      4      1        B  35.544208       0.116162                0.918725   

      depth  outside.sideline  outside.baseline  player.distance.travelled  \
0  6.797621             False             False                   1.467570   
1  2.608708             False              True                   2.311931   
2  9.435749             False             False                   3.903728   
3  3.342180              True             False                   0.583745   
4  5.499119             False             False                   2.333456   

    ...    opponent.depth  opponent.distance.from.center  same

In [29]:
X = df.iloc[:, 1:24].values
Y = df.iloc[:, 26].values
print(X)
print(Y)
print(X.shape)
print(Y.shape)

[[1 'B' 35.51504197 ... 'F' 0.445317963 False]
 [2 'B' 33.38264003 ... 'B' 0.43243397299999997 False]
 [1 'B' 22.3166902 ... 'F' 0.397537762 True]
 ...
 [2 'F' 16.90628902 ... 'B' 0.966185615 False]
 [2 'F' 15.19971253 ... 'B' 0.887608207 False]
 [1 'F' 30.67953985 ... 'B' 0.562388497 True]]
['UE' 'FE' 'FE' ... 'W' 'W' 'UE']
(10000, 23)
(10000,)


### Pre-processing

In [30]:
# Encoding categorical data.
labelEncoder = LabelEncoder()
for col in [1,6,7,19,20,22]:
    X[:, col] = labelEncoder.fit_transform(X[:, col])
    
from sklearn.feature_selection import SelectPercentile, f_classif
p = SelectPercentile(f_classif, percentile=90)
X = p.fit_transform(X, Y)

# Categorical representation: ['FE', 'UE', 'W']
Y = keras.utils.to_categorical(labelEncoder.fit_transform(Y), num_classes=3)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

# Feature Scaling.
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [31]:
# Check shapes.
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)
print("X_test: ", X_test.shape)
print("Y_test: ", Y_test.shape)

X_train:  (8000, 20)
Y_train:  (8000, 3)
X_test:  (2000, 20)
Y_test:  (2000, 3)


### Model

In [32]:
def classifier():

    model = Sequential()

    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    return model

In [33]:
model = classifier()

# Model summary.
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 64)                1344      
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 195       
Total para

![title](img/graph.png)

### Train

In [34]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(X_train, Y_train,
          epochs=125,
          batch_size=25,
          callbacks=[tensorboard])


Epoch 1/125
8000/8000 [==============================] - 1s 77us/step - loss: 0.7817 - acc: 0.6528
Epoch 2/125
8000/8000 [==============================] - 0s 62us/step - loss: 0.5712 - acc: 0.7749
Epoch 3/125
8000/8000 [==============================] - 0s 60us/step - loss: 0.5211 - acc: 0.7928
Epoch 4/125
8000/8000 [==============================] - 0s 60us/step - loss: 0.4987 - acc: 0.8035
Epoch 5/125
8000/8000 [==============================] - 0s 58us/step - loss: 0.4944 - acc: 0.8079
Epoch 6/125
8000/8000 [==============================] - 0s 53us/step - loss: 0.4656 - acc: 0.8184
Epoch 7/125
8000/8000 [==============================] - 0s 57us/step - loss: 0.4572 - acc: 0.8244
Epoch 8/125
8000/8000 [==============================] - 0s 61us/step - loss: 0.4471 - acc: 0.8275
Epoch 9/125
8000/8000 [==============================] - 0s 60us/step - loss: 0.4420 - acc: 0.8275
Epoch 10/125
8000/8000 [==============================] - 0s 61us/step - loss: 0.4306 - acc: 0.8366
Epoch 11/

8000/8000 [==============================] - 1s 64us/step - loss: 0.2963 - acc: 0.8876
Epoch 84/125
8000/8000 [==============================] - 1s 65us/step - loss: 0.3064 - acc: 0.8827
Epoch 85/125
8000/8000 [==============================] - 1s 64us/step - loss: 0.3028 - acc: 0.8850
Epoch 86/125
8000/8000 [==============================] - 1s 66us/step - loss: 0.3009 - acc: 0.8839
Epoch 87/125
8000/8000 [==============================] - 1s 66us/step - loss: 0.3014 - acc: 0.8864
Epoch 88/125
8000/8000 [==============================] - 1s 68us/step - loss: 0.2973 - acc: 0.8857
Epoch 89/125
8000/8000 [==============================] - 1s 66us/step - loss: 0.2921 - acc: 0.8855
Epoch 90/125
8000/8000 [==============================] - 1s 75us/step - loss: 0.3011 - acc: 0.8917
Epoch 91/125
8000/8000 [==============================] - 1s 63us/step - loss: 0.2990 - acc: 0.8837
Epoch 92/125
8000/8000 [==============================] - 1s 67us/step - loss: 0.2987 - acc: 0.8862
Epoch 93/125


## Loss

![loss](img/loss.png)

## Accuracy

![acc](img/acc.png)

### Evaluation

In [35]:
print('Testing:')
score = model.evaluate(X_test, Y_test)
print(model.metrics_names[0], ': ', score[0], '\n', model.metrics_names[1], ': ',score[1])


Testing:
2000/2000 [==============================] - 0s 87us/step
loss :  0.33511979794502256 
 acc :  0.8805


## Conclusion

Despite almost no feature engineering, the ANN was still able to achieve around 90% accuracy on the test set which is in my opinion quite acceptable. It is important to remember that the distinction between an unforced and forced error is based on the nature of the incoming shot and a human judgment about whether the shot was playable or not. As you can imagine, this distinction is not a perfect science.  